In [2]:
import numpy as np 
import pandas as pd
import torch 
from  Data_Loader import DATA_1M
from helper_functions import accuracy_fn, train, create_writer , save_model
from CnnModel import NeuralNetCNN
from sklearn.metrics import (accuracy_score, classification_report,
                              f1_score, recall_score)
from torch.utils.tensorboard import SummaryWriter
import warnings 
warnings.filterwarnings('ignore')

In [5]:
input_size = [250, 500, 1000 ,2000,4000]
for i in range(len(input_size)):

    data = DATA_1M(seconds=30,columns=input_size[i], jump_time =2, n_jumps=1) ; data_fourier = data(Fourier=True, Normalizing= True)
    
    Torch = NeuralNetCNN(columns= data_fourier.shape[1] -1,conv_blocks =1,groupblocks=1)

    data.Spliting(data= data_fourier, random_state= 30, test_size = 0.25, shuffle = True, inplace= False)

    train_dataloader , test_dataloader = data.DataLoaders(batch_size=128, inplace=True)

    experiment_name = f"Fourier_{i}_{data_fourier.shape[1]-1}_inputsize_{len(list(Torch.Cnn.children()))}_Layers_{Torch.count_blocks(Torch.Cnn)}_Blocks"
    
    model_name = experiment_name

    print(f"Experiment Fourier:  {data_fourier.shape[1]-1} input size ,  {len(list(Torch.Cnn.children()))} Layers ,{Torch.count_blocks(Torch.Cnn)} Blocks")
    # torch_pred = Torch.Making_Predictions(model=Torch.Cnn, data_loader= test_dataloader)
            
    # recall_clear = recall_score(data.y_test, torch_pred, average= "weighted", labels=np.array([0]))
    # recall_wifi = recall_score(data.y_test, torch_pred, average= "weighted", labels=np.array([1]))
    # recall_lte = recall_score(data.y_test, torch_pred, average= "weighted", labels=np.array([2]))
    
    # data ={
    #             'Labels': ["CLEAR","WIFI", "LTE"], 
    #             'Recall':[recall_clear,recall_wifi ,recall_lte],
                
    #         }
    # df = pd.DataFrame(data)
    # df = df.set_index('Labels')
    # html_table = df.to_html()
    
    # writer.add_text(f"""input size - {len(list(Torch.Cnn.children()))} Layers ,
    #                        {Torch.count_blocks(Torch.Cnn)} Blocks""", html_table)
    
    Torch.training_loop(model= Torch.Cnn,
        data_loader_train= train_dataloader,
        data_loader_test= test_dataloader,
        optimizer= Torch.optimizer,
        loss_fn= Torch.loss_fn,
        device= Torch.device,
        accuracy_fn= accuracy_fn,
        writer= create_writer(experiment_name=experiment_name,
                              model_name= model_name
                              ),
        epochs= 20 )
    ## Salvando o modelo
    save_filepath = f"Model _{model_name}.pth"
    save_model(model=Torch.Cnn,
               target_dir="models",
               model_name=save_filepath)
    

tamanho da memória ocupada :2297.97 MB
X_train shape: (900000, 250) float64
X_Test shape: (300000, 250) float64
y_train shape: (900000,) float64
y_test shape: (300000,) float64

--------
X_train device: cpu
X_Test device: cpu
y_train device: cpu
y_test device: cpu
Valor 0: 400000 ocorrência(s)- 0.33%
Valor 1: 400000 ocorrência(s)- 0.33%
Valor 2: 400000 ocorrência(s)- 0.33%
Dataset :  (1200000, 251)
X torch.Size([128, 250]) y torch.Size([128])
----------------

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x0000021E40093F10>, <torch.utils.data.dataloader.DataLoader object at 0x0000021E73B40850>)
Length of train dataloader: 7032 batches of 128
Length of test dataloader: 2344 batches of 128
Experiment Fourier:  250 input size ,  3 Layers ,4 Blocks
[INFO] Created SummaryWriter, saving to: runs\2023-06-03\Fourier_0_250_inputsize_3_Layers_4_Blocks\CNN_Model...
Training on cpu


  0%|          | 0/20 [00:00<?, ?it/s]

 Epoch: 0
---------
Train loss: 1.02721 | Train accuracy: 47.84%
Test loss: 1171.05037 | Test accuracy: 51.63%


  5%|▌         | 1/20 [01:08<21:32, 68.02s/it]

 Epoch: 1
---------
Train loss: 0.99441 | Train accuracy: 52.15%
Test loss: 1161.16515 | Test accuracy: 52.48%


...

Test loss: 85.68003 | Test accuracy: 37.32%
 Epoch: 18
---------
Train loss: 0.61978 | Train accuracy: 93.34%


 95%|█████████▌| 19/20 [06:43<00:22, 22.03s/it]

Test loss: 85.93691 | Test accuracy: 37.23%
 Epoch: 19
---------
Train loss: 0.61587 | Train accuracy: 93.65%


100%|██████████| 20/20 [07:05<00:00, 21.27s/it]

Test loss: 85.75927 | Test accuracy: 37.31%
[INFO] Saving model to: models\Model _CNN_Model.pth


In [8]:
input_size = [250, 500, 1000 ,2000,4000]
writer = SummaryWriter()

for i in range(len(input_size)):

    data = DATA_1M(seconds=30,columns=input_size[i], jump_time =2, n_jumps=1) ; data_fourier = data(Fourier=True, Normalizing= True)
    
    Torch = NeuralNetCNN(columns= data_fourier.shape[1] -1,conv_blocks =2,groupblocks=2)

    data.Spliting(data= data_fourier, random_state= 30, test_size = 0.25, shuffle = True, inplace= False)

    train_dataloader , test_dataloader = data.DataLoaders(batch_size=128, inplace=True)

    experiment_name = f"Fourier_{i}_{data_fourier.shape[1]-1}_inputsize_{len(list(Torch.Cnn.children()))}_Layers_{Torch.count_blocks(Torch.Cnn)}_Blocks"
    
    model_name = 'CNN_2Step'

    print(f"Experiment Fourier:  {data_fourier.shape[1]-1} input size ,  {len(list(Torch.Cnn.children()))} Layers ,{Torch.count_blocks(Torch.Cnn)} Blocks")
    # torch_pred = Torch.Making_Predictions(model=Torch.Cnn, data_loader= test_dataloader)
            
    # recall_clear = recall_score(data.y_test, torch_pred, average= "weighted", labels=np.array([0]))
    # recall_wifi = recall_score(data.y_test, torch_pred, average= "weighted", labels=np.array([1]))
    # recall_lte = recall_score(data.y_test, torch_pred, average= "weighted", labels=np.array([2]))
    
    # data ={
    #             'Labels': ["CLEAR","WIFI", "LTE"], 
    #             'Recall':[recall_clear,recall_wifi ,recall_lte],
                
    #         }
    # df = pd.DataFrame(data)
    # df = df.set_index('Labels')
    # html_table = df.to_html()
    
    # writer.add_text(f"""input size - {len(list(Torch.Cnn.children()))} Layers ,
    #                        {Torch.count_blocks(Torch.Cnn)} Blocks""", html_table)
    
    Torch.training_loop(model= Torch.Cnn,
        data_loader_train= train_dataloader,
        data_loader_test= test_dataloader,
        optimizer= Torch.optimizer,
        loss_fn= Torch.loss_fn,
        device= Torch.device,
        accuracy_fn= accuracy_fn,
        writer= create_writer(experiment_name=experiment_name,
                              model_name= model_name
                              ),
        epochs= 20 )
    ## Salvando o modelo
    save_filepath = f"Model _{model_name}.pth"
    save_model(model=Torch.Cnn,
               target_dir="models",
               model_name=save_filepath)
    

tamanho da memória ocupada :2297.97 MB
X_train shape: (900000, 250) float64
X_Test shape: (300000, 250) float64
y_train shape: (900000,) float64
y_test shape: (300000,) float64

--------
X_train device: cpu
X_Test device: cpu
y_train device: cpu
y_test device: cpu
Valor 0: 400000 ocorrência(s)- 0.33%
Valor 1: 400000 ocorrência(s)- 0.33%
Valor 2: 400000 ocorrência(s)- 0.33%
Dataset :  (1200000, 251)
X torch.Size([128, 250]) y torch.Size([128])
----------------

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x0000021E40BC45D0>, <torch.utils.data.dataloader.DataLoader object at 0x0000021E40B9B6D0>)
Length of train dataloader: 7032 batches of 128
Length of test dataloader: 2344 batches of 128
Experiment Fourier:  250 input size ,  4 Layers ,8 Blocks
[INFO] Created SummaryWriter, saving to: runs\2023-06-03\Fourier_0_250_inputsize_4_Layers_8_Blocks\CNN_2Step...
Training on cpu


  0%|          | 0/20 [00:00<?, ?it/s]

 Epoch: 0
---------
Train loss: 1.03923 | Train accuracy: 46.36%
Test loss: 1187.18905 | Test accuracy: 49.94%


  5%|▌         | 1/20 [01:40<31:41, 100.10s/it]

 Epoch: 1
---------
Train loss: 1.00233 | Train accuracy: 51.10%
Test loss: 1174.85165 | Test accuracy: 51.14%


...

 Epoch: 18
---------
Train loss: 0.68678 | Train accuracy: 86.33%
Test loss: 84.80511 | Test accuracy: 39.50%


 95%|█████████▌| 19/20 [07:55<00:24, 24.90s/it]

 Epoch: 19
---------
Train loss: 0.68329 | Train accuracy: 86.61%
Test loss: 85.05243 | Test accuracy: 39.12%


100%|██████████| 20/20 [08:20<00:00, 25.05s/it]

[INFO] Saving model to: models\Model _CNN_2Step.pth


In [9]:
input_size = [250, 500, 1000 ,2000,4000]

for i in range(len(input_size)):

    data = DATA_1M(seconds=30,columns=input_size[i], jump_time =2, n_jumps=1) ; data_fourier = data(Fourier=True, Normalizing= True)
    
    Torch = NeuralNetCNN(columns= data_fourier.shape[1] -1,conv_blocks =4,groupblocks=3)

    data.Spliting(data= data_fourier, random_state= 30, test_size = 0.25, shuffle = True, inplace= False)

    train_dataloader , test_dataloader = data.DataLoaders(batch_size=64, inplace=True)

    experiment_name = f"Fourier_{i}_{data_fourier.shape[1]-1}_inputsize_{len(list(Torch.Cnn.children()))}_Layers_{Torch.count_blocks(Torch.Cnn)}_Blocks"
    
    model_name = 'CNN_3STEP'

    print(f"Experiment Fourier:  {data_fourier.shape[1]-1} input size ,  {len(list(Torch.Cnn.children()))} Layers ,{Torch.count_blocks(Torch.Cnn)} Blocks")

    
    Torch.training_loop(model= Torch.Cnn,
        data_loader_train= train_dataloader,
        data_loader_test= test_dataloader,
        optimizer= Torch.optimizer,
        loss_fn= Torch.loss_fn,
        device= Torch.device,
        accuracy_fn= accuracy_fn,
        writer= create_writer(experiment_name=experiment_name,
                              model_name= model_name
                              ),
        epochs= 25)
    ## Salvando o modelo
    save_filepath = f"Model _{model_name}.pth"
    save_model(model=Torch.Cnn,
               target_dir="models",
               model_name=save_filepath)
    

tamanho da memória ocupada :2297.97 MB
X_train shape: (900000, 250) float64
X_Test shape: (300000, 250) float64
y_train shape: (900000,) float64
y_test shape: (300000,) float64

--------
X_train device: cpu
X_Test device: cpu
y_train device: cpu
y_test device: cpu
Valor 0: 400000 ocorrência(s)- 0.33%
Valor 1: 400000 ocorrência(s)- 0.33%
Valor 2: 400000 ocorrência(s)- 0.33%
Dataset :  (1200000, 251)
X torch.Size([64, 250]) y torch.Size([64])
----------------

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x0000021E41E87B10>, <torch.utils.data.dataloader.DataLoader object at 0x0000021E41E85E10>)
Length of train dataloader: 14063 batches of 64
Length of test dataloader: 4688 batches of 64
Experiment Fourier:  250 input size ,  6 Layers ,18 Blocks
[INFO] Created SummaryWriter, saving to: runs\2023-06-04\Fourier_0_250_inputsize_6_Layers_18_Blocks\CNN_3STEP...
Training on cpu


  0%|          | 0/25 [00:00<?, ?it/s]

 Epoch: 0
---------
Train loss: 1.07068 | Train accuracy: 42.20%
Test loss: 2470.25840 | Test accuracy: 45.32%


  4%|▍         | 1/25 [05:29<2:11:45, 329.38s/it]

 Epoch: 1
---------
Train loss: 1.04730 | Train accuracy: 45.93%
Test loss: 2448.16024 | Test accuracy: 46.62%


...

 Epoch: 23
---------
Train loss: 0.74310 | Train accuracy: 80.50%
Test loss: 168.30542 | Test accuracy: 39.72%


 96%|█████████▌| 24/25 [17:04<00:43, 43.68s/it]

 Epoch: 24
---------
Train loss: 0.73781 | Train accuracy: 81.05%
Test loss: 169.28927 | Test accuracy: 38.99%


100%|██████████| 25/25 [17:48<00:00, 42.73s/it]

[INFO] Saving model to: models\Model _CNN_3STEP.pth


In [10]:
input_size = [250, 500, 1000 ,2000,4000]

for i in range(len(input_size)):

    data = DATA_1M(seconds=30,columns=input_size[i], jump_time =2, n_jumps=1) ; data_fourier = data(Fourier=False, Normalizing= True)
    
    Torch = NeuralNetCNN(columns= data_fourier.shape[1] -1,conv_blocks =1,groupblocks=2)

    data.Spliting(data= data_fourier, random_state= 30, test_size = 0.25, shuffle = True, inplace= False)

    train_dataloader , test_dataloader = data.DataLoaders(batch_size=128, inplace=True)

    experiment_name = f"No_Fourier_{i}_{data_fourier.shape[1]-1}_inputsize_{len(list(Torch.Cnn.children()))}_Layers_{Torch.count_blocks(Torch.Cnn)}_Blocks"
    
    model_name = 'CNN_4STEP'

    print(f"Experiment Fourier:  {data_fourier.shape[1]-1} input size ,  {len(list(Torch.Cnn.children()))} Layers ,{Torch.count_blocks(Torch.Cnn)} Blocks")

    
    Torch.training_loop(model= Torch.Cnn,
        data_loader_train= train_dataloader,
        data_loader_test= test_dataloader,
        optimizer= Torch.optimizer,
        loss_fn= Torch.loss_fn,
        device= Torch.device,
        accuracy_fn= accuracy_fn,
        writer= create_writer(experiment_name=experiment_name,
                              model_name= model_name
                              ),
        epochs= 25)
    ## Salvando o modelo
    save_filepath = f"Model _{model_name}.pth"
    save_model(model=Torch.Cnn,
               target_dir="models",
               model_name=save_filepath)
    

tamanho da memória ocupada :2297.97 MB
X_train shape: (900000, 250) float64
X_Test shape: (300000, 250) float64
y_train shape: (900000,) float64
y_test shape: (300000,) float64

--------
X_train device: cpu
X_Test device: cpu
y_train device: cpu
y_test device: cpu
Valor 0: 400000 ocorrência(s)- 0.33%
Valor 1: 400000 ocorrência(s)- 0.33%
Valor 2: 400000 ocorrência(s)- 0.33%
Dataset :  (1200000, 251)
X torch.Size([128, 250]) y torch.Size([128])
----------------

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x0000021E40F9BC50>, <torch.utils.data.dataloader.DataLoader object at 0x0000021E41908F10>)
Length of train dataloader: 7032 batches of 128
Length of test dataloader: 2344 batches of 128
Experiment Fourier:  250 input size ,  3 Layers ,5 Blocks
[INFO] Created SummaryWriter, saving to: runs\2023-06-04\No_Fourier_0_250_inputsize_3_Layers_5_Blocks\CNN_4STEP...
Training on cpu


  0%|          | 0/25 [00:00<?, ?it/s]

 Epoch: 0
---------
Train loss: 0.81734 | Train accuracy: 72.34%
Test loss: 912.41609 | Test accuracy: 76.32%


  4%|▍         | 1/25 [01:34<37:44, 94.34s/it]

 Epoch: 1
---------
Train loss: 0.77253 | Train accuracy: 77.13%
Test loss: 892.69339 | Test accuracy: 78.16%


...

Test loss: 58.72719 | Test accuracy: 75.48%
 Epoch: 23
---------
Train loss: 0.58235 | Train accuracy: 96.88%
Test loss: 58.82476 | Test accuracy: 75.30%


 96%|█████████▌| 24/25 [08:49<00:22, 22.78s/it]

 Epoch: 24
---------
Train loss: 0.58148 | Train accuracy: 96.97%
Test loss: 58.82968 | Test accuracy: 75.20%


100%|██████████| 25/25 [09:12<00:00, 22.10s/it]

[INFO] Saving model to: models\Model _CNN_4STEP.pth


In [2]:
input_size = [250, 500, 1000 ,2000,4000]

for i in range(len(input_size)):

    data = DATA_1M(seconds=30,columns=input_size[i], jump_time =2, n_jumps=1) ; data_fourier = data(Fourier=False, Normalizing= True)
    
    Torch = NeuralNetCNN(columns= data_fourier.shape[1] -1,conv_blocks =4,groupblocks=2)

    data.Spliting(data= data_fourier, random_state= 30, test_size = 0.3 , shuffle = True, inplace= False)

    train_dataloader , test_dataloader = data.DataLoaders(batch_size=128, inplace=True)

    experiment_name = f"No_Fourier_{i}_{data_fourier.shape[1]-1}_inputsize_{len(list(Torch.Cnn.children()))}_Layers_{Torch.count_blocks(Torch.Cnn)}_Blocks"
    
    model_name = f"CNN_7STEP_No_Fourier_6_Layers_{i}"

    print(f"Experiment Fourier:  {data_fourier.shape[1]-1} input size ,  {len(list(Torch.Cnn.children()))} Layers ,{Torch.count_blocks(Torch.Cnn)} Blocks")

    
    Torch.training_loop(model= Torch.Cnn,
        data_loader_train= train_dataloader,
        data_loader_test= test_dataloader,
        optimizer= Torch.optimizer,
        loss_fn= Torch.loss_fn,
        device= Torch.device,
        accuracy_fn= accuracy_fn,
        writer= create_writer(experiment_name=experiment_name,
                              model_name= model_name
                              ),
        epochs= 25)
    ## Salvando o modelo
    save_filepath = f"Model _{model_name}.pth"
    save_model(model=Torch.Cnn,
               target_dir="models",
               model_name=save_filepath)
    

tamanho da memória ocupada :2297.97 MB
X_train shape: (900000, 250) float64
X_Test shape: (300000, 250) float64
y_train shape: (900000,) float64
y_test shape: (300000,) float64

--------
X_train device: cpu
X_Test device: cpu
y_train device: cpu
y_test device: cpu
Valor 0: 400000 ocorrência(s)- 0.33%
Valor 1: 400000 ocorrência(s)- 0.33%
Valor 2: 400000 ocorrência(s)- 0.33%
Dataset :  (1200000, 251)
X torch.Size([128, 250]) y torch.Size([128])
----------------

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x0000016B01AA8150>, <torch.utils.data.dataloader.DataLoader object at 0x0000016B019A2550>)
Length of train dataloader: 7032 batches of 128
Length of test dataloader: 2344 batches of 128
Experiment Fourier:  250 input size ,  4 Layers ,8 Blocks
[INFO] Created SummaryWriter, saving to: runs\2023-06-04\No_Fourier_0_250_inputsize_4_Layers_8_Blocks\CNN_6STEP_No_Fourier_4_Layers_0...
Training on cpu


  0%|          | 0/25 [00:00<?, ?it/s]

 Epoch: 0
---------
Train loss: 0.82253 | Train accuracy: 71.74%
Test loss: 914.54035 | Test accuracy: 76.22%


  4%|▍         | 1/25 [01:39<39:41, 99.22s/it]

 Epoch: 1
---------
Train loss: 0.77415 | Train accuracy: 76.95%
Test loss: 895.11541 | Test accuracy: 77.88%


...

 Epoch: 23
---------
Train loss: 0.59441 | Train accuracy: 95.66%
Test loss: 59.14930 | Test accuracy: 74.81%


 96%|█████████▌| 24/25 [09:58<00:24, 24.68s/it]

 Epoch: 24
---------
Train loss: 0.59401 | Train accuracy: 95.66%
Test loss: 59.11022 | Test accuracy: 74.76%


100%|██████████| 25/25 [10:23<00:00, 24.94s/it]

[INFO] Saving model to: models\Model _CNN_6STEP_No_Fourier_4_Layers_4.pth


In [4]:
input_size = [200, 250, 300, 400,500 ]

for i in range(len(input_size)):

    data = DATA_1M(seconds=14,columns=input_size[i], jump_time =2, n_jumps=1) ; data_fourier = data(Fourier=True, Normalizing= True)
    
    Torch = NeuralNetCNN(columns= data_fourier.shape[1] -1,conv_blocks =5 ,groupblocks=2)

    data.Spliting(data= data_fourier, random_state= 22, test_size = 0.275, shuffle = True, inplace= False)

    train_dataloader , test_dataloader = data.DataLoaders(batch_size=256, inplace=True)

    experiment_name = f"No_Fourier_{i}_{data_fourier.shape[1]-1}_inputsize_{len(list(Torch.Cnn.children()))}_Layers_{Torch.count_blocks(Torch.Cnn)}_Blocks"
    
    model_name = f"CNN_9STEP_Fourier_10_Layers_{i}"

    print(f"Experiment Fourier:  {data_fourier.shape[1]-1} input size ,  {len(list(Torch.Cnn.children()))} Layers ,{Torch.count_blocks(Torch.Cnn)} Blocks")

    
    Torch.training_loop(model= Torch.Cnn,
        data_loader_train= train_dataloader,
        data_loader_test= test_dataloader,
        optimizer= Torch.optimizer,
        loss_fn= Torch.loss_fn,
        device= Torch.device,
        accuracy_fn= accuracy_fn,
        writer= create_writer(experiment_name=experiment_name,
                              model_name= model_name
                              ),
        epochs= 25)
    ## Salvando o modelo
    save_filepath = f"Model _{model_name}.pth"
    save_model(model=Torch.Cnn,
               target_dir="models",
               model_name=save_filepath)
    

tamanho da memória ocupada :1073.46 MB
X_train shape: (507501, 200) float64
X_Test shape: (192501, 200) float64
y_train shape: (507501,) float64
y_test shape: (192501,) float64

--------
X_train device: cpu
X_Test device: cpu
y_train device: cpu
y_test device: cpu
Valor 0: 233334 ocorrência(s)- 0.33%
Valor 1: 233334 ocorrência(s)- 0.33%
Valor 2: 233334 ocorrência(s)- 0.33%
Dataset :  (700002, 201)
X torch.Size([64, 200]) y torch.Size([64])
----------------

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x000001B25CF3A850>, <torch.utils.data.dataloader.DataLoader object at 0x000001B1CC98F3D0>)
Length of train dataloader: 7930 batches of 64
Length of test dataloader: 3008 batches of 64
Experiment Fourier:  200 input size ,  7 Layers ,17 Blocks
[INFO] Created SummaryWriter, saving to: runs\2023-06-04\No_Fourier_0_200_inputsize_7_Layers_17_Blocks\CNN_9STEP_Fourier_10_Layers_0...
Training on cpu


  0%|          | 0/25 [00:00<?, ?it/s]

 Epoch: 0
---------
Train loss: 1.09263 | Train accuracy: 37.07%
Test loss: 1680.66330 | Test accuracy: 36.19%


  4%|▍         | 1/25 [03:04<1:13:36, 184.03s/it]

 Epoch: 1
---------
Train loss: 1.06611 | Train accuracy: 42.36%
Test loss: 1835.60422 | Test accuracy: 33.16%


...

 Epoch: 3
---------
Train loss: 1.01873 | Train accuracy: 49.55%
Test loss: 1835.60422 | Test accuracy: 33.16%


 16%|█▌        | 4/25 [12:18<1:04:41, 184.82s/it]

 Epoch: 4
---------


 16%|█▌        | 4/25 [15:04<1:19:08, 226.12s/it]


KeyboardInterrupt: 